In [3]:
%matplotlib inline

# Load the modules
import pickle
import math

import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt

# Reload the data
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  train_features = pickle_data['train_dataset']
  train_labels = pickle_data['train_labels']
  valid_features = pickle_data['valid_dataset']
  valid_labels = pickle_data['valid_labels']
  test_features = pickle_data['test_dataset']
  test_labels = pickle_data['test_labels']
  del pickle_data  # Free up memory


print('Data and modules loaded.')

Data and modules loaded.


In [8]:
# Data has been normalized
total_samples = len(train_features)

# Parameters
learning_rate = 0.008
num_output = 10
epochs = 100
batch_size = 256
steps_per_epoch = int(np.ceil(total_samples / batch_size))

In [9]:
# Cleaning Data
#valid_labels, test_labels = np.array(valid_labels, np.int64), np.array(test_labels, np.int64)
train_data = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
val_features, test_features = np.array(valid_features, np.float32), np.array(test_features, np.float32)

# Data has been normalized

In [10]:
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [11]:
features = train_features.shape[1]

In [12]:
W = tf.Variable(tf.random.normal([features, num_output], name="weights"))
B = tf.Variable(tf.zeros([num_output], name="bias"))

optimizer = tf.optimizers.SGD(learning_rate)

# Model
def model(X):
    return tf.nn.softmax(tf.add(tf.matmul(X, W), B))

def cross_entropy(y_pred, y_true):
    # It has been one-hot encoded before storing as pickle
    #y_true = tf.one_hot(y_true, depth=num_output)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), 1))

def accuracy(y_pred, y_true):
    if len(y_true.shape) > 1 and y_true.shape[1] > 1:
        y_true = tf.argmax(y_true, axis=1)
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def run_optimizer(X, Y):
    with tf.GradientTape() as g:
        logit = model(X)
        loss = cross_entropy(logit, Y)

    gradients = g.gradient(loss, [W, B])
    optimizer.apply_gradients(zip(gradients, [W, B]))
    return None

def batch_data(X, Y, batch_size):
    output_data = []
    sample_size = len(X)
    for step in range(0, sample_size, batch_size):
        start = batch_size * step
        end = batch_size + start
        batch_X = X[start:end]
        batch_Y = Y[start:end]
        yield batch_X, batch_Y

In [13]:
# Training
for epoch in range(1, epochs + 1):
    for step, (batch_X, batch_Y) in enumerate(train_data.take(steps_per_epoch), 1):
        run_optimizer(batch_X, batch_Y)

    val_pred = model(valid_features)
    val_loss = cross_entropy(val_pred, valid_labels)
    val_acc = accuracy(val_pred, valid_labels)
    train_pred = model(batch_X)
    train_loss = cross_entropy(train_pred, batch_Y)
    train_acc = accuracy(train_pred, batch_Y)
    print(f"Epoch: {epoch}, train_loss {train_loss}, train_accuracy {train_acc}, val_loss {val_loss}, val_accuracy {val_acc}")

Epoch: 1, train_loss 9.039234161376953, train_accuracy 0.18359375, val_loss 8.769306182861328, val_accuracy 0.23253333568572998
Epoch: 2, train_loss 6.908698081970215, train_accuracy 0.3203125, val_loss 6.341052055358887, val_accuracy 0.3834666609764099
Epoch: 3, train_loss 5.318723201751709, train_accuracy 0.4765625, val_loss 5.422524929046631, val_accuracy 0.47813332080841064
Epoch: 4, train_loss 5.640497207641602, train_accuracy 0.46875, val_loss 4.951755046844482, val_accuracy 0.527999997138977
Epoch: 5, train_loss 4.5839409828186035, train_accuracy 0.59375, val_loss 4.6686906814575195, val_accuracy 0.5651999711990356
Epoch: 6, train_loss 4.251703262329102, train_accuracy 0.5859375, val_loss 4.480134963989258, val_accuracy 0.5894666910171509
Epoch: 7, train_loss 4.863069534301758, train_accuracy 0.5625, val_loss 4.347428798675537, val_accuracy 0.6062666773796082
Epoch: 8, train_loss 3.965963125228882, train_accuracy 0.6640625, val_loss 4.247061252593994, val_accuracy 0.620400011539

In [14]:
test_pred = model(test_features)
test_loss = cross_entropy(test_pred, test_labels)
test_acc = accuracy(test_pred, test_labels)
print(f"Test_loss {test_loss}, Test_accuracy {test_acc}")

Test_loss 0.8740248084068298, Test_accuracy 0.8393999934196472
